HW5-1: iris classification problem (tf.keras)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import shutil

# 1. 清除舊日誌和 TensorBoard 緩存
log_dir = "logs/tfkeras"
cache_dir = os.path.expanduser('~/.tensorboard-info')

# 刪除舊日誌和緩存
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)  # 清除舊 TensorBoard 日誌

if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)  # 清除 TensorBoard 緩存檔案

# 2. 加載 Iris 資料集
data = load_iris()
X = data.data
y = data.target

# 3. 資料標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 4. One-hot 編碼
y = tf.keras.utils.to_categorical(y, 3)

# 5. 資料集切分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. 定義模型
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

# 7. 編譯模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 8. TensorBoard 設置 (覆蓋模式)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

# 9. 訓練模型
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,
                    batch_size=16,
                    callbacks=[tensorboard_callback])

# 10. 評估模型
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# 11. TensorBoard 啟動命令
print(f"啟動 TensorBoard 命令: tensorboard --logdir={log_dir}")

Epoch 1/50
8/8 [==============================] - 1s 46ms/step - loss: 1.0841 - accuracy: 0.3167 - val_loss: 0.8766 - val_accuracy: 0.7667
Epoch 2/50
8/8 [==============================] - 0s 12ms/step - loss: 0.8130 - accuracy: 0.7000 - val_loss: 0.6692 - val_accuracy: 0.8000
Epoch 3/50
8/8 [==============================] - 0s 11ms/step - loss: 0.6783 - accuracy: 0.7250 - val_loss: 0.5425 - val_accuracy: 0.8333
Epoch 4/50
8/8 [==============================] - 0s 11ms/step - loss: 0.5953 - accuracy: 0.7583 - val_loss: 0.4649 - val_accuracy: 0.8333
Epoch 5/50
8/8 [==============================] - 0s 12ms/step - loss: 0.5228 - accuracy: 0.7833 - val_loss: 0.4147 - val_accuracy: 0.8333
Epoch 6/50
8/8 [==============================] - 0s 11ms/step - loss: 0.4792 - accuracy: 0.8083 - val_loss: 0.3784 - val_accuracy: 0.8667
Epoch 7/50
8/8 [==============================] - 0s 11ms/step - loss: 0.4420 - accuracy: 0.8250 - val_loss: 0.3490 - val_accuracy: 0.9000
Epoch 8/50
8/8 [===========

HW5-1: iris classification problem (pytorch)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.tensorboard import SummaryWriter
import os
import shutil

# 1. 清除舊日誌
log_dir = "logs/pytorch"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)  # 移除舊 TensorBoard 日誌
os.makedirs(log_dir)

# 2. 資料準備
data = load_iris()
X = data.data
y = data.target

# 標準化資料
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 資料切分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 轉為 Tensor
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# 建立 DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 3. 定義模型 
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IrisModel().to(device)

# 4. 設定損失函數與優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 5. TensorBoard 設定
writer = SummaryWriter(log_dir=log_dir)

# 6. 訓練模型
epochs = 50
for epoch in range(epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
    
    # 記錄訓練數據
    train_acc = correct / total
    writer.add_scalar('Loss/train', train_loss / len(train_loader), epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    
    # 驗證模型
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
    
    val_acc = correct / total
    writer.add_scalar('Loss/val', val_loss / len(test_loader), epoch)
    writer.add_scalar('Accuracy/val', val_acc, epoch)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


#  啟動 TensorBoard
print(f"啟動 TensorBoard 命令: tensorboard --logdir={log_dir}")

Epoch 1/50, Train Loss: 8.3048, Train Acc: 0.4083, Val Loss: 1.8062, Val Acc: 0.7000
Epoch 2/50, Train Loss: 6.9780, Train Acc: 0.6583, Val Loss: 1.3869, Val Acc: 0.7333
Epoch 3/50, Train Loss: 5.1680, Train Acc: 0.7333, Val Loss: 0.9087, Val Acc: 0.8667
Epoch 4/50, Train Loss: 3.3617, Train Acc: 0.8833, Val Loss: 0.5843, Val Acc: 0.9333
Epoch 5/50, Train Loss: 2.4860, Train Acc: 0.9000, Val Loss: 0.4192, Val Acc: 0.9000
Epoch 6/50, Train Loss: 1.9125, Train Acc: 0.9167, Val Loss: 0.2872, Val Acc: 1.0000
Epoch 7/50, Train Loss: 1.4931, Train Acc: 0.9500, Val Loss: 0.2122, Val Acc: 1.0000
Epoch 8/50, Train Loss: 1.1427, Train Acc: 0.9500, Val Loss: 0.1675, Val Acc: 1.0000
Epoch 9/50, Train Loss: 0.8859, Train Acc: 0.9500, Val Loss: 0.1366, Val Acc: 0.9667
Epoch 10/50, Train Loss: 0.7383, Train Acc: 0.9583, Val Loss: 0.1383, Val Acc: 0.9667
Epoch 11/50, Train Loss: 0.6284, Train Acc: 0.9500, Val Loss: 0.1184, Val Acc: 0.9667
Epoch 12/50, Train Loss: 0.6802, Train Acc: 0.9583, Val Loss: 0

HW5-1: iris classification problem (pytorch lightning)

In [3]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torchmetrics import Accuracy
from torch.utils.tensorboard import SummaryWriter
import os
import shutil

# 1. 清除舊日誌
log_dir = "logs/lightning"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)  # 移除舊 TensorBoard 日誌
os.makedirs(log_dir)

# 2. 資料準備
data = load_iris()
X = data.data
y = data.target

# 標準化資料
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 資料切分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 轉為 Tensor
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# 建立 DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 3. 定義模型
class IrisModel(LightningModule):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 3)
        )
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = Accuracy(task="multiclass", num_classes=3)
        self.val_acc = Accuracy(task="multiclass", num_classes=3)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.train_acc(y_hat, y)

        # 記錄 TensorBoard
        self.log("Loss/train", loss, on_step=False, on_epoch=True)
        self.log("Accuracy/train", acc, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.val_acc(y_hat, y)

        # 記錄 TensorBoard
        self.log("Loss/val", loss, on_step=False, on_epoch=True)
        self.log("Accuracy/val", acc, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.01)

# 4. 設定 Callback
checkpoint_callback = ModelCheckpoint(
    monitor="Loss/val",
    dirpath="checkpoints/",
    filename="best-checkpoint",
    save_top_k=1,
    mode="min",
)

early_stop_callback = EarlyStopping(
    monitor="Loss/val",
    patience=10,
    mode="min"
)

# 5. 訓練與驗證
model = IrisModel()

trainer = Trainer(
    max_epochs=50,
    log_every_n_steps=10,
    callbacks=[checkpoint_callback, early_stop_callback],
    default_root_dir=log_dir,
    enable_progress_bar=True
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

# 6. 啟動 TensorBoard
print(f"啟動 TensorBoard 命令: tensorboard --logdir={log_dir}")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | Sequential         | 193    | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
---------------------------------------------------------
193       Trainable params
0         Non-trainable params
193       Total params
0.001     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


c:\Users\jing5\anaconda3\envs\555\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\jing5\anaconda3\envs\555\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\jing5\anaconda3\envs\555\lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 49: 100%|██████████| 8/8 [00:00<00:00, 91.41it/s, v_num=0] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 8/8 [00:00<00:00, 86.47it/s, v_num=0]
啟動 TensorBoard 命令: tensorboard --logdir=logs/lightning
